# Notebook 000: Project Environment & Dependency Management

### Table of Contents 

- [Notebook 000: Project Environment & Dependency Management](#notebook-000-project-environment--dependency-management)
    - [Table of Contents](#table-of-contents)
  - [1. Imports and Initial Checks](#1-imports-and-initial-checks)
  - [2. Guidance on Creating Virtual Environments](#2-guidance-on-creating-virtual-environments)
    - [Environment Setup Instructions (Recommended: Perform in Terminal)](#environment-setup-instructions-recommended-perform-in-terminal)
      - [Option A: Using `venv` (standard Python)](#option-a-using-venv-standard-python)
      - [Option B: Using `conda`](#option-b-using-conda)
  - [3. Creation of `requirements.txt`](#3-creation-of-requirementstxt)
  - [4. Install Packages from requirements.txt](#4-install-packages-from-requirementstxt)

This notebook serves three main purposes:
1.  **Environment Introspection:** Helps verify the Python environment and `pip` being used by the current Jupyter kernel.
2.  **Environment Setup Guidance:** Provides instructions for creating a dedicated virtual environment (recommended).
3.  **`requirements.txt` Management:**
    * Generates a `requirements.txt` file from the current environment.
    * Provides a command to install packages from an existing `requirements.txt` file.

**IMPORTANT:** For installing packages or freezing requirements for *this specific project*, ensure this notebook's kernel is set to your project's dedicated virtual environment (e.g., `.venv_scdb`, `scdb_conda_env`).

## 1. Imports and Initial Checks

In [1]:
import os
import sys
import platform # To check OS for platform-specific commands
import shutil # For file operations like moving/archiving
from datetime import datetime # For timestamping archived files

print("--- Current Environment Details ---")

# Shows the path to the Python interpreter the kernel is using
# This SHOULD point to the python.exe or python binary within virtual environment
#print(f"Python Executable: {sys.executable}")

# Shows the paths Python searches for modules (site-packages will be in here)
# You should see paths related to your virtual environment listed prominently
# print("\nPython Search Path (sys.path):")
# for path_item in sys.path:
#     print(f"  - {path_item}")

# Displays detailed information about the installed 'pip' package
# Check its 'Location' to confirm it's part of your virtual environment
print("\n--- Pip Details (run in a new cell if '%' magic is preferred) ---")
print("Run '%pip show pip' in a new cell to see pip version and location.")
#%pip show pip

# Returns a string representing the Current Working Directory (CWD)
# This is where relative file paths are resolved from by default.
#print(f"\nCurrent Working Directory (os.getcwd()): {os.getcwd()}")

--- Current Environment Details ---

--- Pip Details (run in a new cell if '%' magic is preferred) ---
Run '%pip show pip' in a new cell to see pip version and location.


## 2. Guidance on Creating Virtual Environments

### Environment Setup Instructions (Recommended: Perform in Terminal)

It is **highly recommended** to create and activate a dedicated virtual environment for this project *from your terminal* **BEFORE** launching Jupyter Lab/Notebook for the first time. This isolates project dependencies.

---
#### Option A: Using `venv` (standard Python)
1.  Open your terminal (Git Bash, PowerShell, CMD, or your system's terminal).
2.  Navigate to your project's root directory (e.g., `H:\...\scdb-case-timing-prediction`).
3.  Create the virtual environment (e.g., named `.venv_scdb`):
    `python -m venv .venv_scdb`
4.  Activate the environment:
    * Windows (Git Bash/PowerShell): `source .venv_scdb/Scripts/activate`
    * Windows (CMD): `.venv_scdb\Scripts\activate.bat`
    * macOS/Linux: `source .venv_scdb/bin/activate`
    *(Your terminal prompt should change to indicate the active environment, e.g., `(.venv_scdb)`)*
5.  Install `ipykernel` into this new environment:
    `pip install ipykernel pandas numpy matplotlib seaborn scikit-learn xgboost hyperopt jupyterlab` (Add other core packages)
6.  Register this environment as a Jupyter kernel:
    `python -m ipykernel install --user --name=.venv_scdb --display-name "Python (.venv_scdb)"`
7.  Now, start Jupyter Lab/Notebook from this activated terminal:
    `jupyter lab`
8.  When you open any project notebook, select the "Python (.venv_scdb)" kernel from the Kernel menu.

---
#### Option B: Using `conda`
1.  Open your Anaconda Prompt or terminal.
2.  Create the conda environment (e.g., named `scdb_env` with Python 3.9):
    `conda create --name scdb_env python=3.9 -y`
3.  Activate the environment:
    `conda activate scdb_env`
4.  Install `ipykernel` and other core packages:
    `conda install ipykernel pandas numpy matplotlib seaborn scikit-learn xgboost -c conda-forge -y`
    `pip install hyperopt` (if not easily available on conda-forge for your setup)
5.  Register this environment as a Jupyter kernel:
    `python -m ipykernel install --user --name=scdb_conda_env --display-name "Python (scdb_conda_env)"`
6.  Now, start Jupyter Lab/Notebook from this activated terminal:
    `jupyter lab`
7.  When you open any project notebook, select the "Python (scdb_conda_env)" kernel.
---

## 3. Creation of `requirements.txt`

In [2]:
# Variable to decide if you want to generate the requirements.txt file
generate_requirements = True

In [3]:
import os
import shutil # Make sure shutil is imported
from datetime import datetime # Make sure datetime is imported

# --- Path Configuration ---
# Determine current notebook directory and project root.
current_notebook_dir = os.getcwd()
print(f"Notebook is currently running in: {current_notebook_dir}")

# Logic to determine project root and relative path for requirements.txt
if os.path.basename(current_notebook_dir).lower() == 'notebooks':
    project_root_dir_for_paths = os.path.abspath(os.path.join(current_notebook_dir, ".."))
    pip_freeze_path_display = '../requirements.txt' # Path for %pip freeze command
else: # Assuming notebook is in project root
    project_root_dir_for_paths = current_notebook_dir
    pip_freeze_path_display = 'requirements.txt'   # Path for %pip freeze command

requirements_file_abs_path = os.path.join(project_root_dir_for_paths, 'requirements.txt')
archive_folder_abs_path = os.path.join(project_root_dir_for_paths, 'archive')

print(f"Target 'requirements.txt' absolute path: {requirements_file_abs_path}")
print(f"Target 'archive' folder absolute path: {archive_folder_abs_path}")

# # Initialize generate_requirements. This is above.
# generate_requirements = False

# Ensure archive folder exists (create if not)
if not os.path.exists(archive_folder_abs_path):
    try:
        os.makedirs(archive_folder_abs_path)
        print(f"Created archive folder: {archive_folder_abs_path}")
    except Exception as e:
        print(f"Could not create archive folder at '{archive_folder_abs_path}'. Archiving will be skipped. Error: {e}")

# Check for existing requirements.txt and prompt for overwrite
if os.path.exists(requirements_file_abs_path):
    print(f"File '{requirements_file_abs_path}' already exists.")
    user_response = input("Do you want to overwrite it? (Yes/No): ").strip().lower() # Changed prompt to y/N
    if user_response == 'Yes': # Check for 'Yes'
        generate_requirements = True
        # Archive the old requirements.txt
        if os.path.exists(archive_folder_abs_path): # Check if archive folder actually exists or was created
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            archive_file_name = f"requirements_{timestamp}.archive.txt" # Suffix for archived file
            archive_destination = os.path.join(archive_folder_abs_path, archive_file_name)
            try:
                # Use absolute paths for file system operations for robustness
                shutil.move(requirements_file_abs_path, archive_destination)
                print(f"Old '{requirements_file_abs_path}' moved to '{archive_destination}'.")
            except Exception as e:
                print(f"Could not archive old requirements.txt. Error: {e}")
        else:
            print(f"Archive folder '{archive_folder_abs_path}' not found/creatable. Old file will not be archived.")
        print(f"Proceeding to generate/overwrite '{requirements_file_abs_path}'.")
    else:
        # generate_requirements was initialized to False, so it remains False if user types anything other than 'Yes'
        print(f"Skipping generation/update of '{requirements_file_abs_path}'.")
else:
    print(f"File '{requirements_file_abs_path}' does not exist. Will create it.")
    generate_requirements = True # Set to True to generate the new file

if generate_requirements:
    print(f"\nACTION: Please run the following command in the NEXT notebook cell to generate/update '{requirements_file_abs_path}':")
    # The path used by %pip freeze should be relative to the notebook's CWD (what os.getcwd() returns)
    print(f"%pip freeze > {pip_freeze_path_display}")
else:
    print(f"\nGeneration of '{requirements_file_abs_path}' was skipped by user.")

Notebook is currently running in: h:\000_Projects\01_GitHub\05_PythonProjects\Twitter\notebooks
Target 'requirements.txt' absolute path: h:\000_Projects\01_GitHub\05_PythonProjects\Twitter\requirements.txt
Target 'archive' folder absolute path: h:\000_Projects\01_GitHub\05_PythonProjects\Twitter\archive
File 'h:\000_Projects\01_GitHub\05_PythonProjects\Twitter\requirements.txt' already exists.
Skipping generation/update of 'h:\000_Projects\01_GitHub\05_PythonProjects\Twitter\requirements.txt'.

ACTION: Please run the following command in the NEXT notebook cell to generate/update 'h:\000_Projects\01_GitHub\05_PythonProjects\Twitter\requirements.txt':
%pip freeze > ../requirements.txt


In [ ]:
# Uncomment the following line to run the pip freeze command directly in this cell
#%pip freeze > ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


## 4. Install Packages from requirements.txt 

In [6]:
os.getcwd()

'h:\\000_Projects\\01_GitHub\\05_PythonProjects\\Twitter\\notebooks'

In [7]:
# Make sure 'requirements.txt' file is in the correct location relative to this notebook.

# If this notebook (000_PackageInstallation.ipynb) is in the main project root:
#%pip install -r requirements.txt

# If this notebook is inside a 'notebooks' subfolder:
#%pip install -r ../requirements.txt

print("To install packages, uncomment the appropriate '%pip install -r ...' line above and run this cell.")

To install packages, uncomment the appropriate '%pip install -r ...' line above and run this cell.


Installation of packages finished !